# 데이터 불러오기

In [ ]:
import os
import pandas as pd
from scipy.io import arff

folder_path = './ISCX-VPN-NonVPN-2016 CSVs/'
all_df = {}
all_source = {}

for sub_folder in os.listdir(folder_path):
  full_path = os.path.join(folder_path, sub_folder)
  if os.path.isdir(full_path):
    all_folder_df = []
    all_folder_source = []
    for filename in os.listdir(full_path):
      if filename.endswith('.arff'):
        file_path = os.path.join(full_path, filename)
        extracted = filename.replace('TimeBasedFeatures-Dataset-', '').replace('.arff', '')
        try:
          data, _ = arff.loadarff(file_path)
          df = pd.DataFrame(data)
          df['class1'] = df['class1'].str.decode('utf-8')
          all_folder_df.append(df)
          all_folder_source.append(extracted)
        except Exception as e:
          print(f"⚠️ 에러 발생: {sub_folder}/{extracted} → {e}")
          continue
    all_df[sub_folder] = all_folder_df
    all_source[sub_folder] = all_folder_source
  
for sub_folder in all_df:
  print(f"=========={sub_folder}==========")
  for df, source in zip(all_df[sub_folder], all_source[sub_folder]):
    shape = f"({df.shape[0]:<5}, {df.shape[1]})"
    print(f"{source:<12} : {shape}")
print("====================================")

: 

# 데이터 매핑 함수 선언
## (범주형 데이터 숫자형으로 변환)

In [ ]:
# mapping
def col_mapping(df_work: pd.DataFrame):
  non_numeric_cols = df_work.select_dtypes(exclude=['number']).columns
  total = {}
  for col in non_numeric_cols:
    normal = {}
    reverse = {}
    idx = 0
    for uni in df[col].unique():
      if (pd.isna(uni)): continue
      normal[uni] = idx
      reverse[idx] = uni
      idx += 1
    total[col] = {"normal": normal, "reverse": reverse}
  return total

def numeric_mapping(df_work: pd.DataFrame, total: dict, reverse: bool):
  cla = "normal"
  if (reverse):
    for col in total:
      df_work[col] = df_work[col].round()
    cla = "reverse"
  for col in total:
    df_work[col] = df_work[col].map(total[col][cla])

## 중복 데이터 처리

In [ ]:
def drop_duplicates(df_work: pd.DataFrame):
  start = len(df_work)
  
  df_work.drop_duplicates(inplace=True)
  
  finish = len(df_work)
  print(f"중복 데이터 처리 : {start} - {start-finish} -> {finish}")

## 누락 데이터 처리

In [ ]:
def processing_missing(df_work: pd.DataFrame):
  from sklearn.impute import KNNImputer

  start_missing = df_work.isnull().sum()
  start_missing = start_missing[start_missing > 0].to_dict()

  total = col_mapping(df_work)
  numeric_mapping(df_work, total, False)

  numeric_cols = df_work.select_dtypes(include='number').columns
  df_numeric = df_work[numeric_cols]

  imputer = KNNImputer(n_neighbors=3)
  df_imputed_numeric = pd.DataFrame(imputer.fit_transform(df_numeric), columns=numeric_cols, index=df_numeric.index)

  df_work[numeric_cols] = df_imputed_numeric
  
  numeric_mapping(df_work, total, True)
  
  finish_missing = df_work.isnull().sum()
  finish_missing = finish_missing[finish_missing > 0].to_dict()
  print(f"누락 데이터 처리 후: {start_missing} -> {finish_missing}")

## 이상치 처리

In [ ]:
def processing_outlier(df_work: pd.DataFrame):
  first = len(df_work)

  total = col_mapping(df_work)
  numeric_mapping(df_work, total, False)

  numeric_cols = df_work.select_dtypes(include='number').columns

  outlier_indices = set()
  for col in numeric_cols:
    Q1 = df_work[col].quantile(0.25)
    Q3 = df_work[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df_work[(df_work[col] < lower_bound) | (df_work[col] > upper_bound)].index
    outlier_indices.update(outliers)

  df_work.drop(index=outlier_indices, inplace=True)
  df_work.reset_index(drop=True, inplace=True)

  numeric_mapping(df_work, total, True)

  print(f"이상치 처리 후: {first} - {len(outlier_indices)} = {len(df_work)}")

# 데이터 전처리

In [ ]:
# 데이터 젙처리
all_df_preprocess = {}
for sub_folder in all_df:
  all_folder_df_preprocess = []
  print(f"=========={sub_folder}==========")
  for df, source in zip(all_df[sub_folder], all_source[sub_folder]):
    df_preprocess = df.copy()
    print("=====================================")
    print(f"{source} - Preprocess")
    drop_duplicates(df_preprocess)
    processing_missing(df_preprocess)
    processing_outlier(df_preprocess)
    all_folder_df_preprocess.append(df_preprocess)
  print("=====================================")
  all_df_preprocess[sub_folder] = all_folder_df_preprocess

## 데이터 전처리 후 shape 변화

In [ ]:
for sub_folder in all_df_preprocess:
  print(f"=========={sub_folder}==========")
  for df, df_preprocess, source in zip(all_df[sub_folder], all_df_preprocess[sub_folder], all_source[sub_folder]):
    before = f"({df.shape[0]:<5}, {df.shape[1]})"
    after = f"({df_preprocess.shape[0]:<5}, {df_preprocess.shape[1]})"
    print(f"{source:<15} : {before} -> {after}")
print("====================================")